![sslogo](https://github.com/stratascratch/stratascratch.github.io/raw/master/assets/sslogo.jpg)

# Data Cleaning with Pandas Tutorial

To edit this notebook:
- Save the notebook by selecting `Download .ipynb` from the `File` tab
- Go to [Colaboratory](https://colab.research.google.com/) and upload the notebook from the `File` tab
- Alternatively, you can import the notebook to your Google Drive and select `Open with` when you right-click. Select `Colaboratory` or `+ Connect more apps` to install Colaboratory first

## Install the Database Module

The code below installs a postgres database module to allow our notebook to connect to the Strata Scratch database



In [ ]:
!pip install psycopg2

## Import Required Modules

Import a few required modules that enables us to query data and perform analytics

In [ ]:
import numpy as np
import pandas as pd
import psycopg2 as ps

## Connect to Strata Scratch

Make sure to enter your username and database password. Your database password is not the same as your login password. You can find your database password in the Profile tab once logged into Strata Scratch.

In [ ]:
host_name = 'db-strata.stratascratch.com'
dbname = 'db_strata'
port = '5432'
user_name = '' #enter username
pwd = '' #enter your database password found in the profile tab in Strata Scratch

try:
    conn = ps.connect(host=host_name,database=dbname,user=user_name,password=pwd,port=port)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

## Pull Data From Strata Scratch

#### Enter SQL code below to pull the dataset you're interested in

If you get an error, it likely means that the connection timed out. Try connecting to Strata Scratch again before executing the code below.

A list of datasets is found in SQL LAB in Strata Scratch.

In [ ]:
#Write SQL below to pull datasets
cur = conn.cursor()
cur.execute("""
            SELECT *  FROM datasets.nfl_combine;
            """)
data = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
conn.commit()

#create the pandas dataframe
data = pd.DataFrame(data)
data.columns = colnames

#close the connection
cur.close()

## Check To See If Your Pulled The Dataset

Your dataset should be in a pandas dataframe named `data`

In [ ]:
data.head()

# Data Cleaning Tips & Tricks Using Pandas

### Fill missing values for college with 'No College'

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html

`fillna` method when given a single value like in this example fills all missing values in the dataframe with that value.

When given a dictionary to the value parameter it replaces according to that dictionary.

`inplace=True` makes the dataframe change permenant in the same dataframe

In [ ]:
data.fillna(value='No College', inplace=True)

### Remove the players that have null values for the pick

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html

Dropna will drop rows according to some null value criteria.

That criteria is defined via the `subset` parameter.

`how="any"` means delete row if at least one value is null.

`how="all"` means delete row if all values are null.

`inplace=True` makes the dataframe change permenant in the same dataframe

In [ ]:
data.dropna(how='any', subset=['pick'], inplace=True)

### Investigate the unique values in the position column

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.unique.html

The unique method returns all unique items as a numpy array.

In some special cases it also returns a Categories object.

In [ ]:
data.position.unique()

### Replace RB and QB with Running Back and Quarterback

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html

This method will change **all values** in the dataframe using a set of rules of the form:
- Replace to_replace[0] with value[0]
- Replace to_replace[1] with value[1]
- ...

`inplace=True` makes the dataframe change permenant in the same dataframe

In [ ]:
data.position.replace(to_replace=['RB','QB'],value=['Running Back','Quarterback'], inplace=True)
data.head()

### Create dummy values for position

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

In [ ]:
dummy_data = pd.get_dummies(data.position, prefix='Pos')
dummy_data.head()

### Merge the dummy data with the original data set

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

Merge is pandas speak for sql join.

The main parameters for merge are:
- dataframe_left which is `data` in our case
- dataframe_right which is `data` in our case
- how which describes the type of SQL JOIN to perform. Some possible values are "left", "right", "outer", "inner"

With that in place only thing left to decide is what to JOIN on.
- We must choose the join keys for both dataframes.
- One choice is to use the index as we do here (for both dataframes actually)
- Another choice is to use columns which is controlled via the parameters left_on and right_on.
- You must not set both left_on and left_index. Same for right.

In a simplified case when you have columns with same name in both dataframes you can use only the `on` parameter.

In [ ]:
data.merge(dummy_data, how='inner', left_index=True, right_index=True)

### Convert weight from lbs to kg

Remmeber: pandas columns are series but they can do all arithmetic stuff that numpy arrays can do.

In [ ]:
data['weight_kg'] = data.weight/2.2
data.head()

### Capitalize name

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html

Apply is a very versatile function which allows arbitrary data transformations.

The main parameters of apply are the function and the axis.

When dealing with columns as in this example, there is no axis, but when using apply on the whole dataframe we must use either axis=1 which means apply over rows or axis=0 which means apply over columns.

In this example we apply `str.upper` over the values in the name column.

In [ ]:
# need the apply function if you're doing an operation on a column. Python doens't know to apply it to the entire dataset.

data.name.apply(str.upper).head()

### Lambda Functions

Lambda functions are short, compact functions in python. They can have any number of arguments but only one expression. The expression is evaluated and returned. Lambda functions can be used wherever function objects are required.

Example:
```
double = lambda x: x * 2

print(double(5))
```



Output would be `10`

This lambda function will double any value in `x`.

#### Reverse order of first and last name

`lambda x:'{0}, {1}'.format(x['lastname'], x['firstname'])`
- x is now a row (**axis=1**) which is of type Series. we can access firstname as x['firstname'] or x.firstname as in any other series.
- The format method in python replaces {i} with i-th argument. So {0} gets replaced with first argument which is x['lastname'] and {1} will get replaced with x['firstname']
- Thus this function takes a row as input and outputs a string which contains the last name followed by the first name

We apply this function over all rows (**axis=1**) and obtain a series as a result.

We store that series in our dataframe with the assignment.

In [ ]:
data['last_firstname'] = data.apply(lambda x:'{0}, {1}'.format(x['lastname'], x['firstname']), axis=1)
data.head()